In [1]:
from fyers_apiv3 import fyersModel
import os
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
import numpy as np
from datetime import timedelta
import math
from fyers_apiv3.FyersWebsocket import data_ws
from sklearn.linear_model import LinearRegression
with open("abcd.txt",'r') as r:
    access_token=r.read()
client_id = "DZO41L3M36-100" 
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path=os.getcwd())
fyers.get_profile()
def calculate_slope_angle(x, y):
    model = LinearRegression().fit(x, y)
    slope = model.coef_[0]
    angle_deg = math.degrees(math.atan(slope))
    return slope, angle_deg


In [2]:
interval=5
symbol="NSE:NIFTY50-INDEX"
data = {
    "symbol":"NSE:NIFTY50-INDEX",
    "resolution":"30",
    "date_format":"1", 
    "range_from":"2024-03-20",
    "range_to":"2024-05-20",
    "cont_flag":"1",
    "interval":"5",
}
response = fyers.history(data=data)

# Check response structure
if 'candles' in response:
    df = pd.DataFrame(response['candles'])
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.date = (df.date.dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata'))
    df['date'] = df['date'].dt.tz_localize(None)
    #df.set_index('date', inplace=True)
    n=len(df)
else:
    print("Error: 'candles' key not found in response")
    print("Response:", response)
    df = pd.DataFrame()  # Create an empty DataFrame to avoid further errors


In [3]:
import plotly.graph_objects as go

if not df.empty:
    # Filter out weekends
    
    fig = go.Figure(data=[go.Candlestick(x=df.date,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'])])


    """
    # Plot candlesticks for trading hours only
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name='Market Hours'))
    
    """
    # Define time bounds for range breaks
    if symbol[:3] == 'MCX' :
        bounds = [24,9]
    elif interval == '1440':
        bounds = [12,18]
    elif interval == '5':
        bounds = [15.5,9.25]
    elif interval == '15':
        bounds = [15.5,9.25]
    elif interval == '30':
        bounds = [15.75,9.25]
    else :
        bounds = [16,9.25]
    alldays =set(df.date[0]+timedelta(x) for x in range((df.date[len(df.date)-1]- df.date[0]).days))
    missing=sorted(set(alldays)-set(df.date))
    fig.update_xaxes(
        rangeslider_visible=False,
        rangebreaks=[
            dict(bounds=bounds, pattern="hour"),
            dict(values=missing)
        ]
    )

    fig.show()
else:
    print("DataFrame is empty. Cannot proceed with analysis.")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
def detect_consecutive_ibars(df):
    consecutive_ibars = [False] * len(df)
    for i in range(len(df)-1):
        previous_candle = df.iloc[i]

        j = i + 1
        count = 0
        while j < len(df) and \
              (df.iloc[j]['high'] < previous_candle['high']) and \
              (df.iloc[j]['low'] > previous_candle['low']) and \
              (df.iloc[j]['close'] < previous_candle['high']) and \
              (df.iloc[j]['open'] > previous_candle['low']):
            
            count += 1
            j += 1
        
        if count > 3:
            for k in range(i+1, j):
                consecutive_ibars[k] = True
            # Mark the larger candlestick as well
            consecutive_ibars[i] = True
                
    return consecutive_ibars

df['ConsecutiveInsideBars'] = detect_consecutive_ibars(df)


# Plot empty squares around consecutive inside bars
i = 0
while i < len(df):
    if df['ConsecutiveInsideBars'][i]:
        start = i
        while i < len(df) and df['ConsecutiveInsideBars'][i]:
            i += 1
        end = i - 1
        
        # Determine the x0 and x1 values, considering the edge cases
        x0 = df['date'][start - 1] if start > 0 else df['date'][start]
        x1 = df['date'][end + 1] if end < len(df) - 1 else df['date'][end]

        fig.add_shape(type="rect",
                      x0=x0, y0=min(df['low'][start:end+1]),
                      x1=x1, y1=max(df['high'][start:end+1]),
                      line=dict(color="black"),
                      fillcolor="rgba(0,0,0,0)")  # Make the box empty
    else:
        i+=1

# Show the figure
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

def detect_consecutive_ibars(df):
    consecutive_ibars = [False] * len(df)
    for i in range(len(df)-1):
        previous_candle = df.iloc[i]

        j = i + 1
        count = 0
        while j < len(df) and \
              (df.iloc[j]['high'] < previous_candle['high']) and \
              (df.iloc[j]['low'] > previous_candle['low']) and \
              (df.iloc[j]['close'] < previous_candle['high']) and \
              (df.iloc[j]['open'] > previous_candle['low']):
            
            count += 1
            j += 1
        
        if count > 3:
            for k in range(i+1, j):
                consecutive_ibars[k] = True
            # Mark the larger candlestick as well
            consecutive_ibars[i] = True
            # Skip to the end of the current pattern to avoid overlapping boxes
            i = j - 1
                
    return consecutive_ibars

df['ConsecutiveInsideBars'] = detect_consecutive_ibars(df)

# Plot empty squares around consecutive inside bars
i = 0
while i < len(df):
    if df['ConsecutiveInsideBars'][i]:
        start = i
        while i < len(df) and df['ConsecutiveInsideBars'][i]:
            i += 1
        end = i - 1
        
        # Determine the x0 and x1 values, considering the edge cases
        x0 = df['date'][start - 1] if start > 0 else df['date'][start]
        x1 = df['date'][end + 1] if end < len(df) - 1 else df['date'][end]

        fig.add_shape(type="rect",
                      x0=x0, y0=min(df['low'][start:end+1]),
                      x1=x1, y1=max(df['high'][start:end+1]),
                      line=dict(color="black"),
                      fillcolor="rgba(0,0,0,0)")  # Make the box empty

        # Move to the next bar after the current pattern
        i = end + 1
    else:
        i += 1

# Show the figure
fig.show()
